## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bethel,US,41.3712,-73.4140,70.11,93,1,0.00,clear sky
1,1,Severo-Kurilsk,RU,50.6789,156.1250,58.71,95,61,5.64,broken clouds
2,2,Vao,NC,-22.6667,167.4833,69.31,61,58,15.95,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,72.25,78,79,12.64,light rain
4,4,Sola,VU,-13.8833,167.5500,77.67,100,90,0.00,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                237
City                   237
Country                234
Lat                    237
Lng                    237
Max Temp               237
Humidity               237
Cloudiness             237
Wind Speed             237
Current Description    237
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(inplace=True)
preferred_cities_df.head(10)

<ipython-input-7-c322dd0621ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preferred_cities_df.dropna(inplace=True)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Sola,VU,-13.8833,167.5500,77.67,100,90,0.00,overcast clouds
7,7,Kurdzhinovo,RU,43.9884,40.9552,81.79,53,1,6.98,clear sky
14,14,Kouango,CF,4.9934,19.9619,79.12,74,100,5.97,overcast clouds
20,20,Manokwari,ID,-0.8667,134.0833,80.96,72,15,2.04,few clouds
21,21,Dongargaon,IN,20.9667,80.8500,85.53,64,100,18.57,overcast clouds
26,26,Siva,EG,29.2041,25.5195,86.65,34,0,6.02,clear sky
34,34,Sembakung,ID,3.8167,117.2500,85.93,59,71,4.29,broken clouds
37,37,Avarua,CK,-21.2078,-159.7750,77.05,83,75,12.66,broken clouds
42,42,Hasaki,JP,35.7333,140.8333,77.40,84,100,25.72,overcast clouds
48,48,Bilma,NE,18.6853,12.9164,89.67,45,98,9.22,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Sola,VU,77.67,overcast clouds,-13.8833,167.5500,
7,Kurdzhinovo,RU,81.79,clear sky,43.9884,40.9552,
14,Kouango,CF,79.12,overcast clouds,4.9934,19.9619,
20,Manokwari,ID,80.96,few clouds,-0.8667,134.0833,
21,Dongargaon,IN,85.53,overcast clouds,20.9667,80.8500,
26,Siva,EG,86.65,clear sky,29.2041,25.5195,
34,Sembakung,ID,85.93,broken clouds,3.8167,117.2500,
37,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,
42,Hasaki,JP,77.40,overcast clouds,35.7333,140.8333,
48,Bilma,NE,89.67,overcast clouds,18.6853,12.9164,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Sola,VU,77.67,overcast clouds,-13.8833,167.5500,Leumerus Bungalows
7,Kurdzhinovo,RU,81.79,clear sky,43.9884,40.9552,Camping Bolshaya Laba
14,Kouango,CF,79.12,overcast clouds,4.9934,19.9619,
20,Manokwari,ID,80.96,few clouds,-0.8667,134.0833,Swiss Belhotel Manokwari
21,Dongargaon,IN,85.53,overcast clouds,20.9667,80.8500,S r farmhouse


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))